In [2]:
import json
from pathlib import Path
import pandas as pd

# locate a likely PeeringDB dump JSON in the workspace
candidates = list(Path('.').glob('*peeringdb*dump*.json')) + list(Path('.').glob('peeringdb*.json'))
if not candidates:
    raise FileNotFoundError("PeeringDB dump JSON not found. Place a file like 'peeringdb_dump.json' in the working directory.")
filepath = candidates[0]

with filepath.open('r', encoding='utf-8') as f:
    dump = json.load(f)

# extract the net.data section and load into a DataFrame
net_data = dump.get('net', {}).get('data')
if net_data is None:
    raise KeyError("JSON does not contain 'net' -> 'data' structure")

net_df = pd.DataFrame(net_data)

# show a quick preview
net_df.head()

,id,org_id,name,aka,name_long,website,social_media,asn,looking_glass,route_server,...,policy_ratio,policy_contracts,allow_ixp_update,status_dashboard,rir_status,rir_status_updated,logo,created,updated,status
0,1,8897,GTT Communications (AS4436),Formerly known as nLayer Communications,,http://www.gtt.net,"[{'service': 'website', 'identifier': 'http://...",4436,,,...,True,Required,False,None,ok,2024-06-26T04:47:55Z,None,2004-07-28T00:00:00Z,2022-07-27T05:33:22Z,ok
1,2,14,Akamai Technologies,,,https://www.akamai.com/,"[{'service': 'website', 'identifier': 'https:/...",20940,,,...,False,Not Required,False,https://www.akamaistatus.com/,ok,2024-06-26T04:47:55Z,None,2004-07-28T00:00:00Z,2025-10-20T12:16:12Z,ok
2,3,17,DALnet IRC Network,,,http://www.dal.net,"[{'service': 'website', 'identifier': 'http://...",31800,,,...,False,Not Required,False,,ok,2024-06-26T04:47:55Z,None,2004-07-28T00:00:00Z,2025-01-09T13:42:07Z,ok
3,5,9350,Swisscom,IP-Plus,,http://www.swisscom.com,"[{'service': 'website', 'identifier': 'http://...",3303,,telnet://route-server.ip-plus.net,...,True,Required,False,,ok,2024-06-26T04:47:55Z,None,2004-07-28T00:00:00Z,2025-08-12T06:33:30Z,ok
4,6,23,Cox Communications,Cox Communications,,http://www.cox.com/peering,"[{'service': 'website', 'identifier': 'http://...",22773,,,...,False,Required,False,,ok,2024-06-26T04:47:55Z,None,2004-07-28T00:00:00Z,2022-11-28T22:55:17Z,ok


In [3]:
# show all columns in net_df (net_df is defined in a previous cell)
print(f"{len(net_df.columns)} columns:")
pd.DataFrame(net_df.columns, columns=["column"])

41 columns:


,column
0,id
1,org_id
2,name
3,aka
4,name_long
5,website
6,social_media
7,asn
8,looking_glass
9,route_server


In [16]:
def date_stats(dataframe, column_name):
    numeric_data = pd.to_datetime(dataframe[column_name]).astype(int) / 10**9
    stats = {
        'min': pd.to_datetime(numeric_data.min(), unit='s'),
        'max': pd.to_datetime(numeric_data.max(), unit='s'),
        'median': pd.to_datetime(numeric_data.median(), unit='s'),
        'mean': pd.to_datetime(numeric_data.mean(), unit='s'),
        '10-percentile': pd.to_datetime(numeric_data.quantile(0.1), unit='s'),
        '20-percentile': pd.to_datetime(numeric_data.quantile(0.2), unit='s'),
        '30-percentile': pd.to_datetime(numeric_data.quantile(0.3), unit='s'),
        '40-percentile': pd.to_datetime(numeric_data.quantile(0.4), unit='s'),
        '50-percentile': pd.to_datetime(numeric_data.quantile(0.5), unit='s'),
        '60-percentile': pd.to_datetime(numeric_data.quantile(0.6), unit='s'),
        '70-percentile': pd.to_datetime(numeric_data.quantile(0.7), unit='s'),
        '80-percentile': pd.to_datetime(numeric_data.quantile(0.8), unit='s'),
        '90-percentile': pd.to_datetime(numeric_data.quantile(0.9), unit='s'),
    }
    for key, value in stats.items():
        stats[key] = pd.to_datetime(value, unit='s')
    return stats

In [18]:
date_stats(net_df, 'updated')

{'min': Timestamp('2022-07-27 05:33:15'),
 'max': Timestamp('2025-10-22 03:59:18'),
 'median': Timestamp('2023-11-07 10:10:58'),
 'mean': Timestamp('2023-11-29 13:52:58.865152'),
 '10-percentile': Timestamp('2022-07-27 05:34:13'),
 '20-percentile': Timestamp('2022-07-27 05:35:32'),
 '30-percentile': Timestamp('2022-07-27 05:36:47'),
 '40-percentile': Timestamp('2022-11-15 22:55:28'),
 '50-percentile': Timestamp('2023-11-07 10:10:58'),
 '60-percentile': Timestamp('2024-07-17 04:15:26'),
 '70-percentile': Timestamp('2024-12-17 09:17:33'),
 '80-percentile': Timestamp('2025-04-30 20:37:52'),
 '90-percentile': Timestamp('2025-08-12 14:02:49')}

In [20]:
date_stats(net_df, 'created')

{'min': Timestamp('2004-07-28 00:00:00'),
 'max': Timestamp('2025-10-21 16:20:45'),
 'median': Timestamp('2020-02-23 18:55:54'),
 'mean': Timestamp('2019-07-20 04:41:04.483539968'),
 '10-percentile': Timestamp('2013-09-02 19:46:29'),
 '20-percentile': Timestamp('2016-09-27 00:02:01'),
 '30-percentile': Timestamp('2017-10-28 03:10:48'),
 '40-percentile': Timestamp('2019-01-07 11:20:42'),
 '50-percentile': Timestamp('2020-02-23 18:55:54'),
 '60-percentile': Timestamp('2021-03-03 18:15:44'),
 '70-percentile': Timestamp('2022-04-30 16:49:00'),
 '80-percentile': Timestamp('2023-07-09 01:40:14'),
 '90-percentile': Timestamp('2024-08-30 10:48:03')}

In [21]:
date_stats(net_df, 'rir_status_updated')

{'min': NaT,
 'max': Timestamp('2025-10-21 22:55:08'),
 'median': Timestamp('2024-06-26 04:47:55'),
 'mean': Timestamp('2024-07-25 16:02:15.627112704'),
 '10-percentile': Timestamp('2024-06-26 04:47:55'),
 '20-percentile': Timestamp('2024-06-26 04:47:55'),
 '30-percentile': Timestamp('2024-06-26 04:47:55'),
 '40-percentile': Timestamp('2024-06-26 04:47:55'),
 '50-percentile': Timestamp('2024-06-26 04:47:55'),
 '60-percentile': Timestamp('2024-06-26 04:47:55'),
 '70-percentile': Timestamp('2024-06-26 04:47:55'),
 '80-percentile': Timestamp('2024-06-26 04:47:55'),
 '90-percentile': Timestamp('2024-10-25 22:55:08')}